# Preprocessing of IMC data for image segmentation

This script presents the first step of the IMC segmentation pipeline.

To get started, please refer to the [Get started guide](docs/index.md).
For more information on how to use the `imctools` package, please refer to this [documentation](https://bodenmillergroup.github.io/imctools/).

Following this script, you will find an option to download example data.

**Requirements for the input data:**

We recommend to supply the raw data in form of _one zip archive per acquisition session_.
This zip archive should contain the `.mcd` file and all `.txt` files corresponding to one acquisition session.
For working with `.txt` files, please look at the older examples.

To understand the output format, please refer to the [Output](docs/output.md) documentation.

For any feedback please contact: Vito, vito.zanotelli@uzh.ch

## Load required libraries

In the first step, we will load the python libraries that are required for processing the data.

In [1]:
from imctools.converters import ome2analysis
from imctools.converters import ome2histocat
from imctools.converters import mcdfolder2imcfolder
from imctools.converters import exportacquisitioncsv

In [2]:
import sys
import os
import pathlib
import shutil
import re

## (Optional) Obtain the example data

Here, a set of example data is downloaded. 
These can be used to test the pipeline.
When using your own data, please ignore the following code chunk.

In [3]:
import urllib.request

# Specify example input directory
fol_example = pathlib.Path('example_data')
fol_example.mkdir(exist_ok=True)

# Specify urls
urls = [('20170905_Fluidigmworkshopfinal_SEAJa.zip',
         'https://www.dropbox.com/s/awyq9p7n7dexgyt/20170905_Fluidigmworkshopfinal_SEAJa.zip?dl=1') ,
        ('20170906_FluidigmONfinal_SE.zip',
         'https://www.dropbox.com/s/0pdt1ke4b07v7zd/20170906_FluidigmONfinal_SE.zip?dl=1')]

# Download the data to the example input directory
for fn, url in urls:
    fn = fol_example / fn
    if not fn.exists():
        urllib.request.urlretrieve(url, fn)

## Specify the inputs

Here, you will need to specify where the IMC raw data (in form of `.zip` archives) are stored.
The `folders_path_inputs` describes the path where the `.zip` archives are located.
Here, we use the example data to run the pre-processing part of the pipeline.
The `input_file_regexp` parameter specifies a [regular expression](https://docs.python.org/3/library/re.html) to select all files of interest from the input directory.
As an example: if you want to select all files that contain the word "test", you would use the regular expression `'.*test.*.zip'`.

You will also need to specify the location of the panel file (`file_path_panel`) that contains information regarding the column that contains the metal name (`metal_colname`), the column that contains an identifier if the channel should be used for ilastik training (`ilastik_colname`), and the column that contains an identifier if the channel should be used to generate the final stack of channels (`full_colname`). The latter two arguments specify columns which contain 0s or 1s, 1 meaning the indicated channel is used and 0 meaning the channel is not used.

In [10]:
# Path to input folder
folders_path_inputs = ['example_data']

# Regular expression to select files of interest
input_file_regexp = '.*.zip'

# Specifications of the panel file
file_path_panel = 'config/example_panel.csv'
metal_colname = 'Metal Tag'
ilastik_colname = 'ilastik'
full_colname = 'full'

## Specify the outputs

You will need to specify a single folder where the output files of the pipeline are written out to.
Furthermore, we will need to specify a suffix, which will be added to the names of the `.tiff` stacks that are needed for ilastik training and object measurements (see *Generate image stacks for downstream analysis* section below).

In [13]:
# output for OME tiffs
folder_path_base = 'analysis'

# Suffix to be added to output tiff stacks
suffix_full = '_full'
suffix_ilastik = '_ilastik'

In the next chunk, all sub-folders for the output files are generated automatically.

In [17]:
folder_path_base = pathlib.Path(folder_path_base)
folders_path_inputs = [pathlib.Path(f) for f in folders_path_inputs]

# Output sub-folders
folder_path_tiffs = folder_path_base / 'tiffs'
folder_path_ilastik= folder_path_base / 'ilastik'
folder_path_ome= folder_path_base / 'ometiff'
folder_path_cp = folder_path_base / 'cpout'
folder_path_cp_input = folder_path_base / 'cpinp'
folder_path_histocat = folder_path_base / 'histocat'

# Other output
file_path_cp_csv = folder_path_cp / 'panel.csv'
file_path_full_channels_csv = folder_path_cp_input / 'full_channelmeta.csv'
file_path_ilastik_channels_csv = folder_path_cp_input / 'ilastik_channelmeta.csv'
file_path_prob_channels_csv = folder_path_cp_input / 'probab_channelmeta_manual.csv'

for fol in [folder_path_base, folder_path_analysis, folder_path_ilastik,
            folder_path_ome, folder_path_cp, folder_path_histocat,
           folder_path_cp_input]:
    if not fol.exists():
        fol.mkdir(parents=True)

## Convert `.mcd` files to `.ome.tiff` files

In the first step, we will convert the `.zip` archives containing `.mcd` files to folders, which contain `.ome.tiff` files, panoramas and slide overviews using the [mcdfolder_to_imcfolder](https://bodenmillergroup.github.io/imctools/converters/mcdfolder2imcfolder.html#imctools.converters.mcdfolder2imcfolder.mcdfolder_to_imcfolder) function. The `.ome.tiff` files can be read in by commercial and open-source software such as ImageJ using the BioFormats importer. 
We will store files that can't be processed in the `failed_images` list object to allow quality control.
At this stage, only images specified by `input_file_regexp` will be processed.

In [6]:
failed_images = list()
re_fn = re.compile(input_file_regexp)

for fol in folders_path_inputs:
    for fn in fol.glob('*'):
        if re_fn.match(fn.name):
            
            try:
                mcdfolder2imcfolder.mcdfolder_to_imcfolder(input=fn, 
                                                           output_folder=folder_path_ome,
                                                           create_zip=False)
                print(fn.name)
                
            except:
                print(fn.name + " can't be processed!")
                failed_images.append(fn.name)

20170906_FluidigmONfinal_SE.zip
20170905_Fluidigmworkshopfinal_SEAJa.zip


We can also now observe the failed conversions:

In [7]:
print(failed_images)

[]


### Export additional metadata

In the next section, we will export the metadata associated with the different acquisitions using the [export_acquisition_csv](https://bodenmillergroup.github.io/imctools/converters/exportacquisitioncsv.html#imctools.converters.exportacquisitioncsv.export_acquisition_csv) function as well as a copy of the panel file.

In [11]:
# Export acquisition metadata
exportacquisitioncsv.export_acquisition_csv(root_folder=folder_path_ome, 
                                            output_folder=folder_path_cp_input)

# Copy panel to output folder
shutil.copy(file_path_panel, file_path_cp_csv)

PosixPath('analysis/cpout/panel.csv')

## Convert `.ome.tiff` files to `histoCAT` compatible format

In the next step, we will convert the generated `.ome.tiff` files to a format that [histoCAT](https://bodenmillergroup.github.io/histoCAT/) can recognize.
For each acquistion (each `.ome.tiff` file), the [omefolder_to_histocatfolder](https://bodenmillergroup.github.io/imctools/converters/ome2histocat.html#imctools.converters.ome2histocat.omefolder_to_histocatfolder) function call produces one folder that contains single channel tiff files. Here, all channels contained in the `.ome.tiff` files are written out.

In [12]:
for fol in folder_path_ome.iterdir():
    if fol.is_dir():
        ome2histocat.omefolder_to_histocatfolder(input_folder=fol, 
                                                 output_folder=folder_path_histocat)

## Generate image stacks for downstream analyses

Next, we will generate two stacks of multi-channel `.tiff` images:

**1. Full stack:** The full stack contains all channels specified by the "1" entries in the `full_colname` column of the panel file. This stack will be later used to measure cell-specific expression features of the selected channels.

**2. Ilastik stack:** The ilastik stack contains all channels specified by the "1" entries in the `ilastik_colname` column of the panel file. This stack will be used to perform the ilastik training to generate cell, cytoplasm and background probability masks (see [Ilastik training](https://bodenmillergroup.github.io/ImcSegmentationPipeline/ilastik.html)).

The [omefolder_to_analysisfolder](https://bodenmillergroup.github.io/imctools/converters/ome2analysis.html#imctools.converters.ome2analysis.omefolder_to_analysisfolder) function takes several arguments:

`input_folder` specifies the folder containing the `.ome.tiff` files.  
`output_folder` specifies the folder where the `.tiff` stacks should be stored.  
`panel_csv_file` specifies the path to the panel file.  
`analysis_stacks` takes an array of analysis stack definitions in a tuple format (column, suffix). Here, column specifies the panel column containing 0s and 1s indicating which channels to use and suffix spcifies the suffix to add at the end of the output stacks.
`metalcolumn` specifies the panel column that contains the metal tags.

We will first generate the stack definition before calling the converter function.

In [16]:
# Define stacks
list_analysis_stacks =[
    (ilastik_colname, suffix_ilastik),
    (full_colname, suffix_full)]

# Convert ome.tiffs to tiff stacks
ome2analysis.omefolder_to_analysisfolder(input_folder=folder_path_ome, 
                                         output_folder=folder_path_tiffs, 
                                         panel_csv_file=file_path_panel,
                                         analysis_stacks=(list_analysis_stacks), 
                                         metalcolumn=metal_colname)

### Export additional metadata

Finally, we will copy a file that contains the correct order of channels for the exported full stacks and ilastik stacks to the input folder.

In [18]:
fn_full = next(folder_path_analysis.glob(f'*{suffix_full}.csv'))
fn_ilastik = next(folder_path_analysis.glob(f'*{suffix_ilastik}.csv'))

shutil.copy(fn_full, file_path_full_channels_csv)
shutil.copy(fn_ilastik, file_path_ilastik_channels_csv)

PosixPath('analysis/cpinp/ilastik_channelmeta.csv')

We will also generate channel metadata for the probability stack.

In [19]:
probab_meta = ["CellCenter", "CellBorder", "Background"]
with open(file_path_prob_channels_csv, 'w') as f:
    f.write('\n'.join(probab_meta))

# Next steps

This concludes the conversion of the IMC rawdata into usable TIFFs.

The pipelines can be found in the `cp4_pipeline` folder in this repository. They were tested in `cellprofiler 4.0.6).

The next steps are:

### A) Cellprofiler: 1_prepare_ilastik

In this module we prepare the data for Ilastik pixel classification, by first removing strong outlier pixels, then scaling the images 2x and then taking random 500x500 crops to do the train the pixel classifier.

Note: for large datasets 250x250 crops or smaler should suffice!

The following parts of this module need to be adapted:

1) File list: choose all files in the `tiff` subfolder

2) Default Output Folder: Choose the `ilastik` subfolder

No further parts need to be adapted.
In our 16 core computer this step takes ca 5 min for the example dataset.


### B) Ilatik: Train a pixel classifier

This uses the random crops generated in the last step.

1) Make a new `pixel classification project`. -> An example project that works with the example data can be found in the 'analysis' subfolder.

2) Add the `.h5` random crops: Raw data -> Add Seperate Images -> Select all `.h5` images in the `ilastik` subfolder.

3) Proceed to `Feature Selection`

4) Select suitable features (or just everything >= 1 pixels)

5) Proceed to the classification:

    - Add 3 labels:
        - 1: Nuclei
        - 2: Cytoplasma/membrane
        - 3: Background
        - -> For large datasets adding the labels can take a while
    - Start labeling:
        - The box next to `Input Data` can change the channels. What each channel corresponds to can be seen when looking in any of the `..._ilastik.csv` files in the `tiff` folder. The 0 channel correspond to the sum of all channels, very usefull to label the background.
        - Use window leveling change the contrast. Right click on the `Input Data` -> `Adjust Thresholds` is also very usefull
        - Label opiniated: If you see in the nucleus channel that two nuclei are stuck together but have a faint dip in intensity in between, label this as 2: Cytoplasma. Encyrcle nuclei with Cytoplasma
        - Diseable `Live Update` for performance
        - Frequently check the `Uncertainties`: This indicates which pixels the classifier profits most if they are labeled. A well trained classifier has low uncertainty within class regions (e.g. Nuclei) and high uncertainty at class borders (e.g. between nuclei and cytoplasma).

    - If you think the classifier is well trained, export the probabilities:
        - Export Settings -> Source: Probabilities -> Choose Export Image Settings:
            - Convert to datatype: Unsigned Integer 16 bit
            - Renormalize: check
            - Format: Tiff
            - File: leave default
        - Export all: This generates `_Probabilities.tiff` in the `ilastik` folder. They can be checked using any image viewer
            - To generate uncertainty maps (good to identify regions that need training),
            run the `Convert probabilities to uncertainties` section `#For training` below. This will put uncertainties in the uncertainty folder.
            -> Well trained classifiers have low uncertainty (transparent) everywhere but at class borders which should be white.

        - Optional: Train again regions with high uncertainty, then proceed.

        - Batch processing: -> Select raw data files -> select all `_s2.h5` files in the `tiff` folder. (sort by filetype, select all `H5` files).
            -> This step takes a while and is computationally intensive!
            -> Ca 15 min on 10 cores on the example data

        - Optional: use the below probability to uncertainty `#For the data` to convert all proabilities to uncertainties, check if there are any regions of high uncertainty and optionally crop the corresponding image part in imagej and add it to the training data.
        - Note: store the `ilastik` folder with all the random crops and the trained classifier for reproducibility reasons.
        
        - A trained

### C) Cellprofiler: 2_segment_ilastik

This step will segment the probabilities into masks.

Things to adapt:

1) File list: choose again all files from the `tiffs` folder

2) It is important to check the `IdentifyPrimaryObjects` step, if the segmentation settings are suitable!
    This might vary strongly between cell/tissue/training and needs attention! Use the test mode and try various settings.
    Also note the `smooth` step immediately before: This can be also removed, I just happen get good results with this additional step.

3) Also the `MeasureObjectSizeShape` combined with `FilterObjects` is just some personal preference of mine, feel free to change

4) `IdentifySecondaryObjects`: Here th mask is expanded to the full cell.

5) `Rescale objects`: note that our segmentation was done on 2x upscaled images, this scales the masks down again. Note that potentially also the nuclei mask could be scaled down and further exported and used.

6) The `Default Output Path` does not need to be adapted for this module.


Note1: Seperating mask generation from mask measurement adds modularity and is thus highly recommended, as generating masks is one of the most resource intensive steps.


### D) Cellprofiler: 3_measure_mask

This step is not necessary for `HistoCat` only analysis. If `HistoCat` should be used, use the `Generate the histocat folder with masks` section below.

#### 3_measure_mask_basic

This module measures without considering spillover correction.

1) File list: choose again all files from the `tiffs` folder

2) View Output settings: set the `Default output folder` to the `cpout` folder and the
    `Default input folder` to the `cpint` folder.

3) Metadata: update - this will automatically merge the mcd metadata .csv generated earlier in the script with your images.

4) Names and types: click update

5) `Measure Object Intensity Multichannel`: Adapt the channel numbers. Check the `_full.csv` files in the `tiffs` folder to see how many channels the stack have and adapt accordingly.

6) `Measure Image Intensity Multichannel`: Adapt the channel numbers. Check the `_full.csv` files in the `tiffs` folder to see how many channels the stack have and adapt accordingly.

Notes:
- In this pipeline all the intesities are scaled by `1/(2**16)`
- The mapping between channel number c1, c2, c3 corresponds to the position in the `_full.csv`s found in the `tiffs` folder.
- The original acquisition description, acquisition frequencies etc can be found in the `Image.csv` output as `Metdata_...` columns.
- This outputs a lot of measurements that are acutally of little interest - usually we only look at `meanintensity` per channel and cell.
    To reduce the outputs, select in `Export To Spreadsheet` -> `Select Measurements to Export` -> Only the measurements you want (usually all Image measurements and only the `MeanIntensity` fullstack measurements).
- The `FullStack` can also be not measured, as it is almost identical to the `FullStackFiltered`.

#### 3_measure_mask_compensated
This will also have a spillover corrections step - stay tuned!


### E) Pipeline output

The pipeline output is all in the `cpout` folder.

Files and folders:
- Image.csv: Image level metadata
- var_Image.csv: Metadata for the colums in Image.csv.
    This contains also metadata from the IMC such as acquisition coordinates.

- {object}.csv: eg cell.csv, contains cell slice level measurements
- var_{object}.csv: eg var_cell.csv: contains metadata for the object measurements

- panel.csv: a copy of the panel used for the input

- Object relationships.csv: Object neighbourhood and other relationships

- Experiment.csv: Metadata about the actual measurement run (eg pipeline used,...)

## Generate the histocat folder with masks

In [31]:
%%time
suffix_mask = '_mask.tiff'
for fol in folder_path_ome.glob('*'):
    ome2histocat.omefolder_to_histocatfolder(fol, folder_path_histocat,
                                    mask_folder=folder_path_analysis, mask_suffix=suffix_mask, dtype='uint16')

CPU times: user 1.35 s, sys: 1.09 s, total: 2.44 s
Wall time: 17.3 s
